In [30]:
import torch

torch.set_printoptions(threshold=torch.inf)

# 加载序列化的 .pt 文件
file_path = '/data/processed/real3dad/train/starfish_506_bulges_cut.pt'  # 替换为实际的文件路径

import torch, numpy as np, pprint

# 让 numpy / torch 打印不省略
np.set_printoptions(threshold=np.inf, linewidth=200, suppress=True)
try:
    torch.set_printoptions(profile="full")  # PyTorch 全量打印
except TypeError:
    torch.set_printoptions(threshold=float('inf'), linewidth=200)

def to_numpy(x):
    if isinstance(x, np.ndarray):
        return x
    if torch.is_tensor(x):
        return x.detach().cpu().numpy()
    return np.asarray(x)

def print_full(name, arr, as_int=False):
    a = to_numpy(arr)
    if as_int:
        a = a.astype(int)
    print(f"\n[{name}] shape={a.shape}, dtype={a.dtype}")
    # 用 array2string 强制完整展开
    print(np.array2string(a, max_line_width=200, separator=', '))

# 载入 .pt
data = torch.load(file_path, map_location='cpu')

# 元信息
print("\n[meta]")
pprint.pprint(data['meta'], width=120)

# 全量打印各字段
print_full('points', data['points'])                       # (N,3)
print_full('semantic_labels_pp', data['semantic_labels_pp'])   # (N,)
print_full('instance_labels_pp', data['instance_labels_pp'])   # (N,)
print_full('instance_masks', data['instance_masks'])           # (K,N) 布尔
print_full('semantic_labels_inst', data['semantic_labels_inst'])  # (K,)


[meta]
{'category': 'good', 'category_id': 1, 'file': '/data/Real3D-AD-PCD/starfish/test_neo/506_bulges_cut.pcd'}

[points] shape=(8192, 3), dtype=float32
[[109.74467   ,  -5.1041675 , 222.97057   ],
 [ 94.18568   , -17.206057  , 230.31433   ],
 [ 94.15756   , -17.09471   , 230.2543    ],
 [ 94.22339   , -17.044086  , 230.19215   ],
 [ 94.52619   , -17.111273  , 230.13245   ],
 [ 94.07376   , -17.419346  , 230.67426   ],
 [ 94.31412   , -16.933504  , 230.09996   ],
 [ 94.05939   , -16.215376  , 230.00558   ],
 [ 94.27754   , -15.152857  , 229.75684   ],
 [ 94.74051   , -14.126217  , 229.27168   ],
 [ 95.237976  , -12.963232  , 228.8191    ],
 [ 95.571945  , -12.056389  , 228.48195   ],
 [ 95.96125   , -11.129998  , 228.09622   ],
 [ 96.51554   , -10.164607  , 227.68669   ],
 [ 97.05544   ,  -9.340914  , 227.20587   ],
 [ 97.75625   ,  -8.543347  , 226.7443    ],
 [ 98.116104  ,  -7.813375  , 226.45772   ],
 [ 98.29856   ,  -7.9864635 , 225.57515   ],
 [ 98.04056   ,  -7.3022223 , 225.

In [4]:
# GT一致性检查
import numpy as np, torch
from pathlib import Path

root = Path("/data/processed/real3dad")
scene = "205_good_cut"

pt = torch.load(root/"val"/f"{scene}.pt", map_location="cpu")
N_pt = pt["points"].shape[0]

gt = np.loadtxt(root/"instance_gt"/"val"/f"{scene}.txt", dtype=np.int64)
print("N_pt =", N_pt, " N_gt =", gt.shape[0], " equal? ", N_pt == gt.shape[0])
print("unique semantic_id in GT:", np.unique(gt//1000))
print("unique inst_index in GT:", np.unique(gt%1000))

N_pt = 2048  N_gt = 2048  equal?  True
unique semantic_id in GT: [0]
unique inst_index in GT: [0]


In [ ]:
import torch, glob, numpy as np

folders = ['train', 'val', 'test']
base = '/data/processed/real3dad'
all_files = []
for folder in folders:
    all_files.extend(glob.glob(f"{base}/{folder}/*.pt"))

label_counts = {}
bad = []
empty = 0
point_spans = {} 
instance_counts = {}

for fp in all_files:
    data = torch.load(fp, map_location='cpu')
    semantic_pp = np.asarray(data['semantic_labels_pp'])
    if len(semantic_pp) == 0:
        empty += 1
        continue
    if semantic_pp.min() < 0 or semantic_pp.max() > 1:
        bad.append(fp)
    binc = np.bincount(semantic_pp)
    counts = {label: count for label, count in enumerate(binc) if count > 0}
    if counts:
        label_counts[fp] = counts
    # 新增：统计xyz跨度
    pts = np.asarray(data['points'])  # (N,3)
    span = pts.max(axis=0) - pts.min(axis=0)  # (3,)
    point_spans[fp] = span
    # 实例数量
    semantic_labels_inst = np.asarray(data['semantic_labels_inst'])
    instance_counts[fp] = len(semantic_labels_inst)

print("empty samples:", empty)
print("label out of {0,1} files:", len(bad))

# 收集所有出现过的label，按255优先、其余降序
all_labels = set()
for counts in label_counts.values():
    all_labels.update([k for k in counts])
if 255 in all_labels:
    sorted_labels = [255] + sorted([l for l in all_labels if l != 255], reverse=True)
else:
    sorted_labels = sorted(all_labels, reverse=True)

# 文件排序，按255标签数量倒序（无255的文件视为0）
sorted_items = sorted(
    label_counts.items(),
    key=lambda x: x[1].get(255, 0),
    reverse=True
)

# 打印表头
header = "filename".ljust(60) + "".join([f"{l:>8}" for l in sorted_labels]) + "   n_inst   span_x   span_y   span_z"
print(header)
print("=" * len(header))

# 全量打印
for fp, counts in sorted_items:
    row = fp.ljust(60)
    for l in sorted_labels:
        row += f"{counts.get(l, 0):>8}"
    n_inst = instance_counts[fp]
    span = point_spans[fp]  # (3,)
    row += f"{n_inst:8d}{span[0]:9.3f}{span[1]:9.3f}{span[2]:9.3f}"
    print(row)

# 总计输出
total = {l: 0 for l in sorted_labels}
for counts in label_counts.values():
    for label, count in counts.items():
        if label in total:
            total[label] += count
print("\nTotal count for all labels across all files:")
for l in sorted_labels:
    print(f"label {l:>3}: {total[l]}")


empty samples: 0
label out of {0,1} files: 666
examples: ['/data/processed/real3dad/train/fish_336_sink_cut.pt', '/data/processed/real3dad/train/duck_586_bulge_cut.pt', '/data/processed/real3dad/train/starfish_597_sink_cut.pt', '/data/processed/real3dad/train/gemstone_300_bulge.pt', '/data/processed/real3dad/train/toffees_462_bulge_cut.pt']
filename                                                         255       1       0   n_inst   span_x   span_y   span_z
/data/processed/real3dad/val/airplane_356_good.pt               8192       0       0       0   31.722   29.482   19.276
/data/processed/real3dad/val/seahorse_248_good.pt               8192       0       0       0   29.504   26.335   10.007
/data/processed/real3dad/val/car_290_good.pt                    8192       0       0       0   35.260   19.765   17.692
/data/processed/real3dad/val/car_254_good_cut.pt                8192       0       0       0   34.981   19.715   16.881
/data/processed/real3dad/val/chicken_778_good.pt        

In [10]:
import os
import numpy as np

data_dir = "/data/processed/real3dad/val" # 或 test
for fname in os.listdir(data_dir):
    if fname.endswith(".npy") or fname.endswith(".pcd"):
        try:
            data = np.load(os.path.join(data_dir, fname))
            print(fname, "shape:", data.shape)
        except Exception as e:
            print(fname, "load error:", e)